In [125]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import FloatSlider, IntSlider, VBox, interactive_output
from IPython.display import display
from scipy.linalg import expm

Setting up drone dynamics...

In [126]:
g = 9.81
m = 1.0
l = 0.25
Iy = 0.01
T_hover = m * g / 2

A = np.array([[0,1,0,0,0,0],
              [0,0,0,0,g,0],
              [0,0,0,1,0,0],
              [0,0,0,0,0,0],
              [0,0,0,0,0,1],
              [0,0,0,0,0,0]])
B = np.array([[0,0],
              [0,0],
              [0,0],
              [1/m,1/m],
              [0,0],
              [l/Iy,-l/Iy]])
C = np.array([[1,0,0,0,0,0],
              [0,0,1,0,0,0],
              [0,0,0,0,1,0]])

### Ackermann's Formula

### LQR

In [127]:
from scipy.linalg import solve_continuous_are

def lqr(A, B, Q, R):
    P = solve_continuous_are(A, B, Q, R)
    K = np.linalg.inv(R) @ B.T @ P
    return K

In [128]:
Q = np.diag([10, 1, 10, 1, 100, 10])
R = np.eye(2)
K_ct = lqr(A, B, Q, R)

### Reference Tracking

In [ ]:
dt = 0.01
t_final = 10
steps = int(t_final / dt)
time = np.linspace(0, t_final, steps)

# Sliders for reference
slider_x_ref = FloatSlider(min=0, max=5, step=0.1, value=2.0, description='x_ref')
slider_y_ref = FloatSlider(min=0, max=5, step=0.1, value=2.0, description='y_ref')

# RK4‐based simulation function
def simulate_and_plot(x_ref=2.0, y_ref=2.0):
    # state vector: [x, x_dot, y, y_dot, theta, theta_dot]
    x = np.zeros((6, steps))
    hover_vec = np.array([T_hover, T_hover])
    
    def dynamics(xi):
        ref = np.array([x_ref, 0, y_ref, 0, 0, 0])
        err = xi - ref
        u = -K_ct @ err + hover_vec
        u = np.clip(u, 0, None)
        return A @ xi + B @ (u - hover_vec)
    
    for i in range(steps - 1):
        k1 = dynamics(x[:, i])
        k2 = dynamics(x[:, i] + 0.5 * dt * k1)
        k3 = dynamics(x[:, i] + 0.5 * dt * k2)
        k4 = dynamics(x[:, i] +     dt * k3)
        
        x[:, i+1] = x[:, i] + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)
    
    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
    axes[0].plot(time, x[0], label='x')
    axes[0].axhline(x_ref, color='r', linestyle='--', label='x=0')
    axes[0].set_ylabel('X Position (m)')
    axes[0].legend()
    
    axes[1].plot(time, x[2], label='y')
    axes[1].axhline(y_ref, color='r', linestyle='--', label='y=0')
    axes[1].set_ylabel('Y Position (m)')
    axes[1].legend()
    
    axes[2].plot(time, x[4], label='theta')
    axes[2].axhline(y_ref, color='r', linestyle='--', label='theta=0')
    axes[2].set_ylabel('Pitch θ (rad)')
    axes[2].set_xlabel('Time (s)')
    axes[2].legend()

    plt.tight_layout()
    plt.show()

# Link sliders to plot
interactive_plot = interactive_output(simulate_and_plot, {
    'x_ref': slider_x_ref,
    'y_ref': slider_y_ref
})
display(VBox([slider_x_ref, slider_y_ref, interactive_plot]))

### Discretization

In [130]:
def discretize_euler_forward(A, B, C, T):
    A_d = np.eye(A.shape[0]) + T * A
    B_d = T * B
    C_d = C
    return A_d, B_d, C_d

def discretize_euler_backward(A, B, C, T):
    A_d = np.linalg.inv(np.eye(A.shape[0]) - T * A)
    B_d = T * A_d @ B
    C_d = C
    return A_d, B_d, C_d

def discretize_trapezoidal(A, B, C, h):
    I = np.eye(A.shape[0])
    A_d = np.linalg.inv(I - 0.5 * h * A) @ (I + 0.5 * h * A)
    B_d = np.linalg.inv(I - 0.5 * h * A) @ (h * B)
    C_d = C.copy() if C is not None else None
    return A_d, B_d, C_d

def discretize_exact(A, B, C, h):
    n = A.shape[0]
    m = B.shape[1]
    M = np.zeros((n + m, n + m))
    M[:n, :n] = A
    M[:n, n:] = B
    expM = expm(h * M)
    Ad = expM[:n, :n]
    Bd = expM[:n, n:]
    Cd = C.copy() if C is not None else None
    return Ad, Bd, Cd

### DLQR Controller

In [131]:
from scipy.linalg import solve_discrete_are

def dlqr(A_d, B_d, Q, R):
    P = solve_discrete_are(A_d, B_d, Q, R)
    K = np.linalg.inv(B_d.T @ P @ B_d + R) @ (B_d.T @ P @ A_d)
    return K

In [132]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, interactive_output, VBox
from scipy.linalg import solve_discrete_are

# Continuous‐time step (fixed)
dt_ct = 0.01

# Reference sliders
slider_x_ref = FloatSlider(min=0, max=5, step=0.1, value=2.0, description='x_ref')
slider_y_ref = FloatSlider(min=0, max=5, step=0.1, value=2.0, description='y_ref')
# Discrete‐time step slider
slider_dt_dt = FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='dt_dt')

# Discretization methods dictionary
methods = {
    'Euler Forward': discretize_euler_forward,
    'Euler Backward': discretize_euler_backward,
    'Trapezoidal':   discretize_trapezoidal,
    'Exact':         discretize_exact
}

def simulate_and_compare(x_ref=2.0, y_ref=2.0, dt_dt=0.1):
    # final time and time‐grids
    t_final = 10.0
    time_ct = np.arange(0, t_final + dt_ct, dt_ct)
    time_dt = np.arange(0, t_final + dt_dt, dt_dt)

    # reference state
    ref = np.array([x_ref, 0, y_ref, 0, 0, 0])
    hover_vec = np.array([T_hover, T_hover])

    # Continuous‐time with RK4
    x_ct = np.zeros((6, len(time_ct)))
    def f_ct(xi):
        err = xi - ref
        u = -K_ct @ err + hover_vec
        u = np.clip(u, 0, None)
        return A @ xi + B @ (u - hover_vec)

    for i in range(len(time_ct)-1):
        k1 = f_ct(x_ct[:, i])
        k2 = f_ct(x_ct[:, i] + 0.5 * dt_ct * k1)
        k3 = f_ct(x_ct[:, i] + 0.5 * dt_ct * k2)
        k4 = f_ct(x_ct[:, i] +     dt_ct * k3)
        x_ct[:, i+1] = x_ct[:, i] + (dt_ct / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

    # Discrete‐time systems & LQR gains for this dt_dt
    disc_sys = {}
    K_d = {}
    for name, func in methods.items():
        Ad, Bd, _ = func(A, B, C, dt_dt)
        P_d = solve_discrete_are(Ad, Bd, Q, R)
        K_d[name] = np.linalg.inv(Bd.T @ P_d @ Bd + R) @ (Bd.T @ P_d @ Ad)
        disc_sys[name] = (Ad, Bd)

    # Initialize discrete trajectories
    x_dt = {name: np.zeros((6, len(time_dt))) for name in methods}

    # Step through discrete systems
    for i in range(len(time_dt)-1):
        for name in methods:
            Ad, Bd = disc_sys[name]
            err_dt = x_dt[name][:, i] - ref
            u_dt = np.clip(-K_d[name] @ err_dt + hover_vec, 0, None)
            x_dt[name][:, i+1] = Ad @ x_dt[name][:, i] + Bd @ (u_dt - hover_vec)

    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(8, 9), sharex=False)
    for ax, idx, ylabel in zip(axes, [0, 2, 4], ['X Position (m)', 'Y Position (m)', 'Pitch θ (rad)']):
        # continuous
        ax.plot(time_ct, x_ct[idx], 'k-', label='Continuous (RK4)')
        # discrete
        styles = [('C0','--'), ('C1','-.'), ('C2',':'), ('C3','--')]
        for (name, (color, style)) in zip(methods.keys(), styles):
            ax.plot(time_dt, x_dt[name][idx], color=color, linestyle=style, label=name)
        # reference lines
        if idx == 0:
            ax.axhline(x_ref, color='r', linestyle='--', label='ref')
        elif idx == 2:
            ax.axhline(y_ref, color='r', linestyle='--', label='ref')
        if idx in [0, 2]:  # x and y position plots
            ax.set_ylim(0.0, 5.0)
        ax.set_ylabel(ylabel)
        ax.legend(loc='best')

    axes[-1].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.show()

interactive_plot = interactive_output(
    simulate_and_compare,
    {'x_ref': slider_x_ref, 'y_ref': slider_y_ref, 'dt_dt': slider_dt_dt}
)

display(VBox([slider_x_ref, slider_y_ref, slider_dt_dt, interactive_plot]))


### MPC Formulation

In [ ]:
import cvxpy as cp

methods = {
    'Euler Forward': discretize_euler_forward,
    'Euler Backward': discretize_euler_backward,
    'Trapezoidal':    discretize_trapezoidal,
    'Exact':          discretize_exact
}

# ——— Choose which discretization to use ———
disc_method = 'Exact'  # change to one of: 'Euler Forward', 'Euler Backward', 'Trapezoidal', 'Exact'

nx     = A.shape[0]     # number of states
nu     = B.shape[1]     # number of inputs
T_sim  = 100            # number of discrete‐time steps per simulation
dt_def = 0.1            # default dt for computing terminal cost

# ——— Precompute terminal cost P_inf (for MPC) at the default dt ———
Ad_def, Bd_def, _ = methods[disc_method](A, B, C, dt_def)
P_inf = solve_discrete_are(Ad_def, Bd_def, Q, R)

def add_constraints(constraints, x, u, k):
    constraints += [x[4, k] >= -0.2,
                    x[4, k] <= 0.2]
    return constraints

# ——— DLQR simulation ———
def simulate_dlqr(x_ref, y_ref, dt_dt):
    Ad, Bd, _ = methods[disc_method](A, B, C, dt_dt)
    P_d = solve_discrete_are(Ad, Bd, Q, R)
    K_d = np.linalg.inv(Bd.T @ P_d @ Bd + R) @ (Bd.T @ P_d @ Ad)

    ref       = np.array([x_ref, 0, y_ref, 0, 0, 0])
    hover_vec = np.array([T_hover, T_hover])

    x = np.zeros((nx, T_sim + 1))
    for k in range(T_sim):
        err = x[:, k] - ref
        u   = np.clip(-K_d @ err + hover_vec, 0, None)
        x[:, k+1] = Ad @ x[:, k] + Bd @ (u - hover_vec)

    t = np.linspace(0, T_sim * dt_dt, T_sim + 1)
    return x, t

# ——— MPC setup (no additional constraints) ———
def solve_mpc(x0, N, ref):
    # use the latest Ad, Bd from simulate_mpc
    P_T = P_inf  # terminal cost from default dt
    x = cp.Variable((nx, N+1))
    u = cp.Variable((nu, N))
    cost = 0
    constraints = [x[:, 0] == x0]

    for k in range(N):
        cost += cp.quad_form(x[:, k] - ref, Q) + cp.quad_form(u[:, k], R)
        constraints += [x[:, k+1] == Ad @ x[:, k] + Bd @ u[:, k]]
        constraints += add_constraints([], x, u, k)

    cost += cp.quad_form(x[:, N] - ref, P_T)

    prob = cp.Problem(cp.Minimize(cost), constraints)
    prob.solve(solver=cp.OSQP, verbose=False)
    return u[:, 0].value.flatten()

def simulate_mpc(x_ref, y_ref, dt_dt, N):
    global Ad, Bd
    Ad, Bd, _ = methods[disc_method](A, B, C, dt_dt)

    ref       = np.array([x_ref, 0, y_ref, 0, 0, 0])
    hover_vec = np.array([T_hover, T_hover])

    x = np.zeros((nx, T_sim + 1))
    x[:, 0] = np.zeros(nx)  # or your actual initial state

    for k in range(T_sim):
        u0 = solve_mpc(x[:, k], N, ref)
        u  = np.clip(u0 + hover_vec, 0, None)
        x[:, k+1] = Ad @ x[:, k] + Bd @ (u - hover_vec)

    t = np.arange(0, (T_sim+1) * dt_dt, dt_dt)
    return x, t

# ——— Comparison & interactive plotting ———
def simulate_compare(x_ref, y_ref, dt_dt, N):
    x_lqr, t = simulate_dlqr(x_ref, y_ref, dt_dt)
    x_mpc, _ = simulate_mpc(x_ref, y_ref, dt_dt, N)

    fig, axes = plt.subplots(3, 1, figsize=(8, 9), sharex=True)
    labels = ['X Position (m)', 'Y Position (m)', r'Pitch $\theta$ (rad)']
    idxs   = [0, 2, 4]

    for ax, idx, ylabel in zip(axes, idxs, labels):
        ax.plot(t, x_lqr[idx], 'C0-', label='DLQR')
        ax.plot(t, x_mpc[idx], 'C1--', label=f'DMPC (N={N})')
        if idx == 0:
            ax.axhline(x_ref, color='k', linestyle=':', label='ref')
        elif idx == 2:
            ax.axhline(y_ref, color='k', linestyle=':', label='ref')
        if idx in [0, 2]:
            ax.set_ylim(0.0, 5.0)
        ax.set_ylabel(ylabel)
        ax.legend(loc='best')

    axes[-1].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.show()

# ——— Widgets ———
slider_x_ref = FloatSlider(min=0, max=5, step=0.1, value=2.0, description='x_ref')
slider_y_ref = FloatSlider(min=0, max=5, step=0.1, value=2.0, description='y_ref')
slider_dt_dt = FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='dt_dt')
slider_N     = IntSlider(min=1, max=50, step=1, value=10,  description='Horizon N')

interactive_plot = interactive_output(
    simulate_compare,
    {'x_ref': slider_x_ref,
     'y_ref': slider_y_ref,
     'dt_dt': slider_dt_dt,
     'N':     slider_N}
)

display(VBox([slider_x_ref, slider_y_ref, slider_dt_dt, slider_N, interactive_plot]))